Imports to featurize and load data

In [43]:
import os
os.environ["MKL_NUM_THREADS"] = "1" 
os.environ["NUMEXPR_NUM_THREADS"] = "1" 
os.environ["OMP_NUM_THREADS"] = "3"
import pandas as pd
import numpy as np
import sys
sys.path.append("../../")
from DanceProj1.DanceObj import Dance
from DanceProj1.data_proc import get_data, data_to_features


Imports for Automl

In [35]:
import autosklearn.classification
import sklearn.model_selection
from sklearn.model_selection import train_test_split
import sklearn.datasets
import sklearn.metrics

In [36]:
#make sure autosklearn is here
print('autosklearn: %s' % autosklearn.__version__)

autosklearn: 0.15.0


Data loaded as two dataframes of dances by features, one for basic and one for advanced movement sequences.

In [37]:
aistpath = '../../aist_keypoints'      #path to 3d keypoints data
dataBM, dataFM = get_data(aistpath)    #get data
dfBasic, dfAdvanced = data_to_features(dataBM, dataFM)  #get features as dataframes


In [38]:
#make new indexes for Advanced, starting after last index in Basic, for unique index per id
new_index_advanced = range(len(dfBasic.index), len(dfBasic.index)+len(dfAdvanced.index))
dfAdvanced.index = dfAdvanced.index + new_index_advanced
dfAdvanced.head()

,Expandedness,Expandedness_std,Expandednessvel,Expandednessvel_std,Expandednessacc,Expandednessacc_std,Asym_RL_vel,Asym_RL_acc,Asym_RL_jer,Asym_RL_vel_std,...,Autocorr_wrists_prominenceaxz2,Autocorr_wrists_prominenceaxz3,Autocorr_wrists_prominenceaxz_std,Autocorr_ankles_height_axz,Autocorr_ankle_prominenceaxz1,Autocorr_ankle_prominence2axz,Autocorr_ankle_prominence3axz,Autocorr_ankles_prominenceaxz_std,id,Genre
1199,1056.865643,90.082477,1538.228504,958.219385,7666.414735,3900.555376,2839.095566,3202.027147,-8145.973932,6.328799,...,2.798204e+08,2.307739e+08,6.860505e+08,5.295456e+08,8.538254e+08,1.572569e+08,1.038147e+08,3.416593e+08,gBR_sFM_cAll_d05_mBR2_ch09,Break
1201,1031.250205,115.045741,2035.464041,966.684795,10537.243423,4651.213150,-498.891513,248.747443,687.314989,7.118658,...,3.958780e+08,2.190737e+08,6.411089e+08,1.006350e+09,1.683154e+09,5.729446e+08,2.901511e+08,6.012018e+08,gBR_sFM_cAll_d06_mBR5_ch19,Break
1203,1264.589529,654.019537,3468.162657,2929.231708,17493.828054,14958.905140,2277.125602,1744.592252,-1649.130332,30.364511,...,4.582227e+08,1.320478e+07,1.365297e+09,1.813506e+09,2.806314e+09,4.460812e+08,4.391301e+08,1.114266e+09,gBR_sFM_cAll_d05_mBR5_ch14,Break
1205,1100.526992,108.640735,1619.546822,761.437478,8351.133624,3748.230439,1330.346396,-3083.023419,-94451.348976,15.941952,...,4.961852e+08,2.543382e+08,6.197565e+08,7.128766e+08,1.164893e+09,2.019773e+08,1.205257e+08,4.742883e+08,gBR_sFM_cAll_d04_mBR5_ch06,Break
1207,1012.354348,101.102753,1380.734980,483.503378,6921.930683,2232.699927,2425.553893,-3654.077082,2469.866960,19.227707,...,5.221820e+08,3.404664e+08,4.611543e+08,4.908603e+08,7.804250e+08,1.868732e+08,1.483255e+08,2.893171e+08,gBR_sFM_cAll_d06_mBR2_ch16,Break


Test set is to be composed only of Advanced dances. Most of the data is Basic dances, with each choreography performed with slight variations. Below is a custom splitting function (see data_proc) for getting an all-Advanced Test, and a mix of Advanced and Basic for validation.  
   
Here we use 10 dances from each genre (100 total) for Test, 2 from each genre in Validation, and the remaining ~9 Advanced from each in Train. This leaves 14-Basic-per-genre for validation, and ~105-Basic-per-genre for Train.    


In [39]:
from DanceProj1.data_proc import traintestval_split
train, valid, testset = traintestval_split(dfBasic, dfAdvanced, testfrac_adv=.5, testfrac_bas=0, valfrac_adv_nonT=.2, valfrac_bas=.12)

In [40]:
testset.loc[testset.Genre=='House'].head()

,Expandedness,Expandedness_std,Expandednessvel,Expandednessvel_std,Expandednessacc,Expandednessacc_std,Asym_RL_vel,Asym_RL_acc,Asym_RL_jer,Asym_RL_vel_std,...,Autocorr_wrists_prominenceaxz2,Autocorr_wrists_prominenceaxz3,Autocorr_wrists_prominenceaxz_std,Autocorr_ankles_height_axz,Autocorr_ankle_prominenceaxz1,Autocorr_ankle_prominence2axz,Autocorr_ankle_prominence3axz,Autocorr_ankles_prominenceaxz_std,id,Genre
1437,1000.648601,73.820332,1343.749380,396.988939,6762.410723,1851.523248,3948.170219,-337.728363,-89962.131252,8.807390,...,1.661033e+08,1.061372e+08,3.502705e+08,5.885795e+08,9.569760e+08,1.831791e+08,1.428537e+08,3.746380e+08,gHO_sFM_cAll_d21_mHO0_ch15,House
1427,994.107864,71.798051,1701.801858,613.485878,9147.078841,3162.341899,12.964395,2738.133928,-567.279148,16.312285,...,5.431118e+08,4.109055e+08,6.915368e+08,7.968781e+08,1.295409e+09,2.436909e+08,2.117621e+08,5.034792e+08,gHO_sFM_cAll_d20_mHO0_ch08,House
1413,1019.549964,67.637853,1199.334952,318.683441,5983.288153,1409.172889,-5384.010638,-4457.325293,2088.045080,48.090919,...,3.462538e+08,2.481640e+08,3.856348e+08,5.154741e+08,8.742840e+08,2.368795e+08,1.998074e+08,3.095835e+08,gHO_sFM_cAll_d21_mHO2_ch17,House
1443,1024.030967,70.259960,1441.421811,413.551617,7003.828117,1848.404010,-3918.221437,-92.973499,964.023925,27.533582,...,2.076957e+08,1.751806e+08,3.438537e+08,5.657086e+08,9.171720e+08,2.537778e+08,2.178056e+08,3.215413e+08,gHO_sFM_cAll_d21_mHO1_ch16,House
1441,894.002316,52.196779,1124.061655,385.733371,5773.570059,1968.999417,1932.657251,1712.517789,-1216.411734,9.902315,...,1.899963e+08,1.494164e+08,2.972049e+08,4.789456e+08,7.717179e+08,2.619936e+08,1.523348e+08,2.698722e+08,gHO_sFM_cAll_d19_mHO2_ch03,House


The dfs above include ID, for later evaluation. This column should be removed from input.  

traintestval_split also includes genre, which is our target label in this classification task and should accordingly be removed from x and used for y.

In [41]:
test_ids = testset['id']
index_lookup = pd.DataFrame(
               testset.index, columns=['original_index']) #lookup table for original index

X_test = testset.drop(['Genre'], axis=1).drop(['id'], axis=1)
y_test = testset['Genre']

X_valid = valid.drop(['Genre'], axis=1).drop(['id'], axis=1)
y_valid = valid['Genre']

X_train = train.drop(['Genre'], axis=1).drop(['id'], axis=1)
y_train = train['Genre']

In [22]:
# check the shape 
for item in [X_train, X_test, X_valid, y_valid, y_train, y_test]:
  print(item.shape)

(1148, 44)
(100, 44)
(160, 44)
(160,)
(1148,)
(100,)


Let's see how a Perceptron does (skipping validation)

In [28]:
from sklearn.linear_model import Perceptron
preg = Perceptron(tol=.05, max_iter=2000, alpha=.00001).fit(X_train, y_train)
print('Perceptron is at chance: Score =', preg.score(X_test, y_test))

Perceptron is at chance: Score = 0.1


Let's see how a Ridge Classifier does (skipping validation)

In [30]:
#Let's see how a ridge classifier does
from sklearn.linear_model import RidgeClassifier
rreg = RidgeClassifier().fit(X_train, y_train)
print('RidgeClassifier is better than chance, but not great. Score =',rreg.score(X_test, y_test))

RidgeClassifier is better than chance, but not great. Score =  0.34


/home/tbenbaker/.local/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:212: LinAlgWarning: Ill-conditioned matrix (rcond=9.44476e-37): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


Let's see how a Support Vector Classification does

##### Now for auto-sklearn. The two main parameters for auto-sklearn are:   
- time_left_for_this_task, optional (default=3600): Time limit in seconds for the search of appropriate models. By increasing this value, auto-sklearn has a higher chance of finding better models.  
- per_run_time_limit, optional (default=1/10 of time_left_for_this_task): Time limit for a single call to the machine learning model. Model fitting will be terminated if the machine learning algorithm runs over the time limit. Set this value high enough so that typical machine learning algorithms can be fit on the training data.


In [ ]:
# Instantiate an automl classifier model object with time limits
automl = autosklearn.classification.AutoSklearnClassifier(
        time_left_for_this_task=400,
        per_run_time_limit=50
)

# Fit the classifier to training data
automl.fit(X_train, y_train)